In [79]:
from dotenv import load_dotenv
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [80]:
from langchain_ollama import ChatOllama 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough 
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.tools import tool


llm = ChatOllama(model='llama3.2', base_url='http://localhost:11434')
llm.invoke('hi')

AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-01-01T12:07:06.9396456Z', 'done': True, 'done_reason': 'stop', 'total_duration': 9382677800, 'load_duration': 8800891000, 'prompt_eval_count': 26, 'prompt_eval_duration': 371000000, 'eval_count': 8, 'eval_duration': 183000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-a57deff6-61ae-47cf-85b4-5a8605eb2982-0', usage_metadata={'input_tokens': 26, 'output_tokens': 8, 'total_tokens': 34})

In [81]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

llm = ChatGroq(model='llama-3.2-11b-vision-preview')
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001E64DE1C8D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E636C32350>, model_name='llama3-groq-70b-8192-tool-use-preview', model_kwargs={}, groq_api_key=SecretStr('**********'))

## Define Tools

### 1. Tavily

In [82]:
from langchain_community.tools import TavilySearchResults

@tool
def search(query: str) -> str:
    """
    Search the web for realtime and latest information.
    for examples, news, stock market, weather updates etc.
    
    Args:
    query: The search query
    """
    
    search = TavilySearchResults(
        max_results=5,
        search_depth="advanced",
        include_answer=True,
        include_raw_content=True,
    )
    response = search.invoke(query)
    return response

### 2.wikipedia

In [83]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


@tool
def wikipedia_search(query: str) -> str:
    """
    Search Wikipedia for general information. Use this for historical, cultural, or factual queries.

    Args:
        query: The search query.
    """
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    response = wikipedia.invoke(query)
    return response

### 3.Google search 

In [84]:
from langchain_community.utilities import GoogleSerperAPIWrapper
@tool
def google_search_place(query: str) -> str:
    """
    Search Google for information about places. Use this for location-based queries, such as landmarks, cities, or addresses.

    Args:
        query: The search query.
    """
    search = GoogleSerperAPIWrapper(type='places')
    response = search.results(query)
    return response

In [85]:
question ='best place to visit in chennai'

In [86]:
google_search_place(question)

{'searchParameters': {'q': 'best place to visit in chennai',
  'gl': 'us',
  'hl': 'en',
  'type': 'places',
  'num': 10,
  'engine': 'google'},
 'places': [{'position': 1,
   'title': 'Government Museum Chennai',
   'address': 'Pantheon Rd, Egmore, Chennai, Tamil Nadu 600008, India',
   'latitude': 13.0699328,
   'longitude': 80.2567588,
   'rating': 4.3,
   'ratingCount': 17000,
   'category': 'Tourist attraction',
   'phoneNumber': '+91 44 2819 3238',
   'cid': '10738827242996767798'},
  {'position': 2,
   'title': 'Arulmigu Sri Parthasarathyswamy Temple',
   'address': 'Car St, Narayana Krishnaraja Puram, Triplicane, Chennai, Tamil Nadu 600005, India',
   'latitude': 13.0538068,
   'longitude': 80.27695609999999,
   'rating': 4.7,
   'ratingCount': 26000,
   'category': 'Tourist attraction',
   'phoneNumber': '+91 44 2844 2462',
   'website': 'https://parthasarathy.hrce.tn.gov.in/',
   'cid': '12272515067481418528'},
  {'position': 3,
   'title': 'VGP Universal Kingdom',
   'addres

### 4.Health Supplements Retriever

In [87]:
import os 

In [88]:
from langchain_community.document_loaders import PyMuPDFLoader

pdfs = []
for root,dir,files in os.walk("R:/LLM/8.Document_Loaders/rag-dataset"):
    # print(root,dir,files)
    for file in files:
        # print(file)
        if file.endswith(".pdf"):
            pdfs.append(os.path.join(root, file))

In [89]:
docs = []

for doc in pdfs:
    loader = PyMuPDFLoader(doc)
    temp = loader.load()
    docs.extend(temp)

In [90]:
#split the docs into smaller chunks 
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks = text_splitter.split_documents(docs)

In [91]:
#initialize the embedding 
from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(model='nomic-embed-text',base_url='http://localhost:11434')

In [92]:
#let import the chromaDB
from langchain_chroma import Chroma
# create a vector Db
vector_store = Chroma(embedding_function=embeddings,persist_directory='./chroma_langchain_db',collection_name="pdf_collection")

In [93]:
vector_store.add_documents(documents=chunks)

['b2f82c55-4dd2-4812-afe7-043a32950440',
 'd402d805-9175-44d2-8f0f-e632baef2a1e',
 'ba496b1b-4b48-4689-bb95-2ad54e1b3c97',
 '0ddb8a8e-9e16-4969-ab05-f6f04971f03e',
 'acb0658b-de33-4db4-9f34-4e46fddec31d',
 'feeef565-8b6f-4fb3-890e-9a7938da3ddb',
 '737db060-fe45-4746-a049-31df7a6fb9d3',
 '25b77422-f85b-48f6-9500-8a48dcfa1bb7',
 'fd554a4b-d0df-4ae8-b964-b1a84d1e5fdf',
 'c841f2be-a23e-43a7-856a-04025e6d2275',
 'c3c2538a-1cdb-4cd1-ba6d-4da1680a4bb8',
 '4a859b9e-e36d-4f17-b6ea-132566752e06',
 'e5c2402e-32d0-4b91-8c49-a6dc47f0256f',
 'aca461d7-4766-4e86-b305-e0e98c20bd48',
 '23e44301-711d-48bb-b30e-22e0f8fa5fe0',
 '70dbc11d-3704-4ca0-b88a-8eee44e34488',
 '55b9d2f7-ada3-4e78-a9cf-33f683a512db',
 '947d341e-b3cd-44f5-bd75-b81eb4e39faa',
 'd5c73b27-96da-4edc-85a5-40ac5f5ecac2',
 '39d4c040-c046-42cc-a675-c19f8a13315d',
 '2b825924-63cf-45b0-8d6d-8daeaff679c6',
 '880c3fff-056f-4489-8c76-29d6e704cf47',
 '639a5a98-204b-4e0e-bd71-33a2fd89426e',
 'd02b88db-d04d-4b35-8ff4-0d3c3e584a49',
 'aec5b384-35d3-

In [94]:
# vectordb = Chroma(persist_directory='chroma_langchain_db', embedding_function=embeddings)

In [95]:
# retriever = vectordb.as_retriever(
#     search_type = 'mmr',search_kwards ={"k":1,"fetch_k":10}
# )
# question = "how to lose weight?"
# retriever.invoke(question)

In [96]:
retriever = vector_store.as_retriever(
    search_type='mmr',
    search_kwargs={"k": 3, "fetch_k": 10}
)

In [97]:
question = "how to lose weight?"
retriever.invoke(question)

[Document(metadata={'author': '', 'creationDate': '', 'creator': '', 'file_path': 'R:/LLM/8.Document_Loaders/rag-dataset\\health supplements\\1. dietary supplements - for whom.pdf', 'format': 'PDF 1.7', 'keywords': '', 'modDate': 'D:20241021113754Z', 'page': 12, 'producer': 'iLovePDF', 'source': 'R:/LLM/8.Document_Loaders/rag-dataset\\health supplements\\1. dietary supplements - for whom.pdf', 'subject': '', 'title': '', 'total_pages': 17, 'trapped': ''}, page_content='Int. J. Environ. Res. Public Health 2021, 18, 8897\n13 of 24\n3.3. Dietary Supplements and Weight Loss\nThe number of people with excessive weight continues to rise, and ﬁghting obesity\nhas become one of the greatest challenges of contemporary medicine. A person wishing to\nlose weight needs to undertake several difﬁcult life-changes and practice them consistently\n(diet, physical activity, addiction-free). Meanwhile dietary supplements are presented as\na compelling alternative to traditional methods for combatting obe

In [98]:
@tool
def health_supplements(query: str) -> str:
    """Search for information about Health Supplements.
    For any questions about Health and Gym Supplements, you must use this tool!,

    Args:
        query: The search query.
    """
    response = retriever.invoke(query)
    return response

In [99]:
question ='how to lose weights?'

In [100]:
health_supplements.invoke("what is the best supplement for muscle gain?")

[Document(metadata={'author': '', 'creationDate': '', 'creator': '', 'file_path': 'R:/LLM/8.Document_Loaders/rag-dataset\\gym supplements\\1. Analysis of Actual Fitness Supplement.pdf', 'format': 'PDF 1.7', 'keywords': '', 'modDate': 'D:20241021113850Z', 'page': 0, 'producer': 'iLovePDF', 'source': 'R:/LLM/8.Document_Loaders/rag-dataset\\gym supplements\\1. Analysis of Actual Fitness Supplement.pdf', 'subject': '', 'title': '', 'total_pages': 15, 'trapped': ''}, page_content='and non-competitive athletes [1]. One widely known supplement for athletes and fitness\nenthusiasts is protein supplements like protein shakes. Others may be whey protein,\ncaffeine, and creatine, to name a few. These supplements play a crucial role in helping\nindividuals increase muscle mass, recover faster, and enhance their overall performance [2].\nWhey protein stands out as a top-quality protein source, containing more essential amino\nacids than traditional protein sources. Its numerous benefits have made i

## Agent Creation

- The agent is responsible for taking in input and deciding what actions to take
- The Agent does not execute those actions - that is done by the AgentExecutor
- create_tool_calling_agent will call .bind_tools for us under the hood.

In [101]:
from langchain import hub
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

In [102]:
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [103]:
tools = [search,wikipedia_search,google_search_place,health_supplements]
agent = create_tool_calling_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMes

In [104]:
Agent_executor = AgentExecutor(agent=agent,tools=tools,verbose = True)

In [114]:
question = "What are the side effects of taking too much vitamin D?"
response = Agent_executor.invoke({'input': question})
print(response)
print(response['input'])
print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `health_supplements` with `{'query': 'side effects of too much vitamin D'}`


[Document(metadata={'author': '', 'creationDate': '', 'creator': '', 'file_path': 'R:/LLM/8.Document_Loaders/rag-dataset\\health supplements\\1. dietary supplements - for whom.pdf', 'format': 'PDF 1.7', 'keywords': '', 'modDate': 'D:20241021113754Z', 'page': 10, 'producer': 'iLovePDF', 'source': 'R:/LLM/8.Document_Loaders/rag-dataset\\health supplements\\1. dietary supplements - for whom.pdf', 'subject': '', 'title': '', 'total_pages': 17, 'trapped': ''}, page_content='Int. J. Environ. Res. Public Health 2021, 18, 8897\n11 of 24\nimpact of vitamin D supplementation, some meta-analyses of randomized controlled trials\nindicate that vitamin D intake reduces the risk of cancer mortality by 12–13% [90,91].\nHowever, some authors note that due to the low quality of the research, this ﬁnding\nshould be approached with caution [91] and yet others do not conﬁrm any r

In [106]:
question = 'best place to visit in russia?'
response = Agent_executor.invoke({'input': question})



> Entering new AgentExecutor chain...

Invoking: `google_search_place` with `{'query': 'best place to visit in russia'}`


{'searchParameters': {'q': 'best place to visit in russia', 'gl': 'us', 'hl': 'en', 'type': 'places', 'num': 10, 'engine': 'google'}, 'places': [{'position': 1, 'title': "St. Basil's Cathedral", 'address': 'Red Square, 7, Moscow, Russia, 109012', 'latitude': 55.752522899999995, 'longitude': 37.623086799999996, 'rating': 4.8, 'ratingCount': 11000, 'category': 'Tourist attraction', 'phoneNumber': '+7 495 698-33-04', 'website': 'https://shm.ru/museum/hvb/', 'cid': '11875735528249440997'}, {'position': 2, 'title': 'The Moscow Kremlin', 'address': 'Moscow, Russia, 103132', 'latitude': 55.7509544, 'longitude': 37.6175755, 'rating': 4.8, 'ratingCount': 54000, 'category': 'Tourist attraction', 'phoneNumber': '+7 495 697-03-49', 'website': 'https://www.kreml.ru/', 'cid': '12143603839552663144'}, {'position': 3, 'title': 'Red Square', 'address': 'Moscow, Russia', 'latitude

In [107]:
question = "What is the best supplement for muscle gain?"
response = Agent_executor.invoke({'input':question})



> Entering new AgentExecutor chain...
I'm sorry but I do not have the capability to perform this task for you, I am happy to help you with any other queries you may have.

> Finished chain.


In [108]:
question = "what's weather in New York?"
response = Agent_executor.invoke({'input':question})



> Entering new AgentExecutor chain...

Invoking: `search` with `{'query': 'weather in New York'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'New York', 'region': 'New York', 'country': 'United States of America', 'lat': 40.7142, 'lon': -74.0064, 'tz_id': 'America/New_York', 'localtime_epoch': 1735730995, 'localtime': '2025-01-01 06:29'}, 'current': {'last_updated_epoch': 1735730100, 'last_updated': '2025-01-01 06:15', 'temp_c': 8.3, 'temp_f': 46.9, 'is_day': 0, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png', 'code': 1009}, 'wind_mph': 6.0, 'wind_kph': 9.7, 'wind_degree': 275, 'wind_dir': 'W', 'pressure_mb': 999.0, 'pressure_in': 29.51, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 86, 'cloud': 100, 'feelslike_c': 6.7, 'feelslike_f': 44.0, 'windchill_c': 5.1, 'windchill_f': 41.1, 'heatindex_c': 7.8, 'heatindex_f': 46.1, 'dewpoint_c': 7.3, 'dewpoint_f': 45.2, 'vis_km': 11.0, 'vis_miles': 6.0, 'uv': 0.0, '

In [109]:
question = "Teach me about langchain and llm?"
response = Agent_executor.invoke({'input':question})



> Entering new AgentExecutor chain...

Invoking: `wikipedia_search` with `{'query': 'langchain'}`


Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

Page: Milvus (vector database)
Summary: Milvus is a distributed vector database developed by Zilliz. It is available as both open-source software and a cloud service.
Milvus is an open-source project under LF AI & Data Foundation distributed under the Apache License 2.0.



Page: Retrieval-augmented generation
Summary: Retrieval Augmented Generation (RAG) is a technique that grants generative artificial intelligence models information retrieval capabilities. It modifies interactions with a large language model (LLM) so that the model responds t

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Page: Master of Laws
Summary: A Master of Laws (M.L. or LL.M.; Latin: Magister Legum or Legum Magister) is an advanced postgraduate academic degree, pursued by those either holding an undergraduate academic law degree, a professional law degree, or an undergraduate degree in a related subject. In most jurisdictions, the LL.M. is the advanced professional degree for those usually already admitted into legal practice.



Page: Large language model
Summary: A large language model (LLM) is a type of machine learning model designed for natural language processing tasks such as language generation. LLMs are language models with many parameters, and are trained with self-supervised learning on a vast amount of text.
The largest and most capable LLMs are generative pretrained transformers (GPTs). Modern models can be fine-tuned for specific tasks or guided by prompt engineering. These models acquire predictive power regarding syntax, semantics, and ontologies inherent in human language corpora

In [111]:
question = "google headquaters in located in which place?"
response = Agent_executor.invoke({'input':question})



> Entering new AgentExecutor chain...

Invoking: `google_search_place` with `{'query': 'google headquarters location'}`


{'searchParameters': {'q': 'google headquarters location', 'gl': 'us', 'hl': 'en', 'type': 'places', 'num': 10, 'engine': 'google'}, 'places': [{'position': 1, 'title': 'Mountain View', 'address': 'CA', 'latitude': 37.390026399999996, 'longitude': -122.08123040000001, 'category': 'California', 'cid': '8941849000641539995'}], 'credits': 1}The Google headquarters is located in Mountain View, California.

> Finished chain.
